In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input/images'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot  as plt
img = '../all-isic-data-20240629/images/ISIC_0000000.jpg'


In [ ]:
Image = plt.imread(img)
image = plt.imshow(Image)
plt.show()


In [ ]:

import pandas as pd
df = pd.read_csv('../all-isic-data-20240629/metadata.csv',usecols=["isic_id", "patient_id", "benign_malignant"],)
df.shape


In [ ]:

metadata = pd.read_csv('../all-isic-data-20240629/metadata.csv')
metadata.columns
print(df.patient_id.isnull().sum())


In [ ]:
import  os 

len(os.listdir('../all-isic-data-20240629/images/'))
# lấy số luọng file xuất hiện trong tập



In [ ]:

df.columns
metadata = pd.read_csv("../all-isic-data-20240629/metadata.csv", usecols=["isic_id", "patient_id", "benign_malignant"])
metadata.shape


In [ ]:
# cài đặt thư viện
! pip install keras_cv
! pip install keras_cv_attention_models
!pip install tensorflow
%pip install tensorflow-hub
%pip install keras-cv



In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split


In [ ]:

Xtrain = df[['isic_id', 'patient_id']]
ytrain = df['benign_malignant']

# Split the data into training validation and test sets
X_train, X_test, y_train, y_test = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
print(f"Original data shape: {df.shape}")
print(f"Training set shape: {X_train.shape}")
print(f"Prediction set shape: {X_val.shape}")
print(f"Testing set shape: {X_test.shape}")


In [ ]:
df.columns


In [ ]:
#call model vgg16
from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# Load VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:

# Freeze the base model
for layer in base_model.layers: 
    layer.trainable = False 
# Add custom layers on top of the base model    
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))  # Binary classification


In [ ]:

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print ("model summary")
model.summary()


In [ ]:
df.head()


In [ ]:
#train model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,        
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)  



In [ ]:

# Merge X_train with y_train to get the labels in the dataframe
train_df = X_train.copy()
train_df['benign_malignant'] = y_train

# Keep only rows with 'benign' or 'malignant' labels
train_df = train_df[train_df['benign_malignant'].isin(['benign', 'malignant'])]

# Drop rows with missing labels and ensure labels are strings
train_df = train_df.dropna(subset=['benign_malignant'])
# Thêm đuôi .jpg vào isic_id
train_df['isic_id'] = train_df['isic_id'].astype(str) + '.jpg'
train_df['benign_malignant'] = train_df['benign_malignant'].astype(str)



train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='../all-isic-data-20240629/images/',
    x_col='isic_id',
    y_col='benign_malignant',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)
